In [ ]:
import os 
import ants 
import csv
import SimpleITK as sitk 
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from ipywidgets import HBox
from ipywidgets import Output
from IPython.display import display
from ipywidgets import interact
from helpers2 import *
!pip install scikit-image

In [ ]:
#path to project folder 
BASE_DIR = os.path.dirname(os.path.abspath("_file_"))
print (f'project folder = {BASE_DIR}')


#subject id variables 
samples = [
    '(subject_id_code1)','(subject_id_code2)',
    '(subject_id_code3)' '...'
]

#file paths for outputs 
results = []
folder_path = 'folder path to save csv file'
output_folder1="folder path to save registered MRI images"
output_folder2 = "folder path to save registered cavity images" 

In [ ]:
def process_sample(sample_id):
    # Define paths dynamically based on the sample ID
    postop_path = os.path.join(BASE_DIR, 'MRI', 'postop', sample_id, 'anat', f'{sample_id}_3T_postop_T1w.nii.gz')
    preop_path = os.path.join(BASE_DIR, 'MRI', 'preop', sample_id, 'anat', f'{sample_id}_3T_preop_T1w.nii.gz')
    cavitymask_path = os.path.join(BASE_DIR, 'MRI', 'postop', sample_id, 'anat', 'resection_cavity.nii.gz')
    lesionmask_path = os.path.join(BASE_DIR, 'MRI', 'preop', sample_id, 'anat', f'{sample_id}_3T_lesion_MASK.nii.gz')
    print (f'{sample_id}')


    # Image pre-processing
    postop_ants = ants.image_read(postop_path, reorient='IAL')
    postop_ants_arr = postop_ants.numpy()
    postop = explore_3D_array00(postop_ants_arr)
    
    preop_ants = ants.image_read(preop_path, reorient='IAL')
    preop_ants_arr = preop_ants.numpy()
    preop = explore_3D_array00(preop_ants_arr)

    cavitymask_ants = ants.image_read(cavitymask_path, reorient='IAL')
    cavitymask_ants_arr = cavitymask_ants.numpy()
    cavitymask = explore_3D_array00(cavitymask_ants_arr)

    lesionmask_ants = ants.image_read(lesionmask_path, reorient='IAL')
    lesionmask_ants_arr = lesionmask_ants.numpy()
    lesionmask = explore_3D_array00(lesionmask_ants_arr)
    
    print("postoperative MRI")
    display(postop)
    print("preoperative MRI")
    display(preop)
    print("cavity mask")
    display(cavitymask)
    print("lesion mask")
    display(lesionmask)


    # cavity + postop / lesion + preop 
    cavity_postop = ants.mask_image(postop_ants,cavitymask_ants)
    cp = explore_3D_array_with_mask_contour00 (postop_ants.numpy(),cavitymask_ants.numpy())
    lesion_preop = ants.mask_image(preop_ants,lesionmask_ants)
    lp = explore_3D_array_with_mask_contour00 (preop_ants.numpy(),lesionmask_ants.numpy())
    display (cp)
    display (lp)


    # preop + postop registration / axial view 
    transformation = ants.registration(
        fixed=preop_ants,
        moving=postop_ants,
        type_of_transform='SyN',
        verbose=True 
    )
    registered_ants = transformation['warpedmovout']
    preoppostop = explore_3D_array_with_transparent_overlay(arr=preop_ants.numpy(),overlay=registered_ants.numpy())
    display (preoppostop)
    

    # apply transformation on cavity mask / registered cavity mask + preop 
    registered_c_ants = ants.apply_transforms(
        moving = cavitymask_ants,
        fixed = transformation ['warpedmovout'],
        transformlist=transformation['fwdtransforms'],
        verbose=True
    )
    rc = explore_3D_array_with_mask_contour00(
        arr=preop_ants.numpy(),
        mask=registered_c_ants.numpy()
    )
    display (rc)


    # QC metrics 
    preop_arr = preop_ants.numpy()
    registered_arr = registered_ants.numpy()
    difference = np.abs(preop_arr - registered_arr)
    rmse = np.sqrt(np.mean(difference**2))
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

    from skimage.metrics import structural_similarity as ssim
    assert preop_arr.shape == registered_arr.shape
    ssi_values = []
    for i in range(preop_arr.shape[0]):
        slice_ssi = ssim(preop_arr[i, :, :], registered_arr[i, :, :], data_range=preop_arr.max() - preop_arr.min())
        ssi_values.append(slice_ssi)
    vol_ssi=ssim(preop_arr, registered_arr, data_range=preop_arr.max() - preop_arr.min())
    average_ssi = np.mean(ssi_values)
    print(f"Average Structural Similarity Index (SSI) across all slices: {average_ssi:.4f}")
    

# Overlap Percentage + results 
    def calculate_overlap_percentage(cavity_mask, lesion_mask):
        # Ensure the masks are binary
        cavity_mask = (cavity_mask > 0).astype(int)
        lesion_mask = (lesion_mask > 0).astype(int)
        
        # Calculate intersection and union
        intersection = np.sum(cavity_mask * lesion_mask)
        lesion_volume = np.sum(lesion_mask)
        
        if lesion_volume == 0:
            return 0.0  # Avoid division by zero if the lesion mask is empty
        
        # Percentage overlap
        overlap_percentage = (intersection / lesion_volume) * 100
        return overlap_percentage

    registered_c_mask = registered_c_ants.numpy()  
    lesionmask = lesionmask_ants.numpy()                 
    overlap_percentage = calculate_overlap_percentage(registered_c_mask, lesionmask)
    print(f"Overlap Percentage : {overlap_percentage:.2f}%")
    
    prefix = sample_id  # Use the sample ID as a prefix
    ants.image_write(registered_ants, f'{output_folder1}{prefix}_registered.nii.gz')
    ants.image_write(registered_c_ants, f'{output_folder2}{prefix}_registered_cavity.nii.gz')
    
    results.append({
        'Sample ID': sample_id,
        'Overlap Percentage': overlap_percentage,
        'RMSE': rmse,
        'SSI': vol_ssi
    })


# Loop through all the sample IDs and process them
for sample_id in samples:
    process_sample(sample_id)
csv_file_path = os.path.join(folder_path, 'resultfilename.csv')


# After the loop, write the results to a CSV file
with open(csv_file_path, mode='w', newline='') as file:
    # Specify the fieldnames (columns) in the order you want
    fieldnames = ['Sample ID', 
                  'Overlap Percentage', 'RMSE', 'SSI']
    
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()  # Write the header row
    writer.writerows(results) 
